# Vectara

## Overview

[Vectara](https://vectara.com/) is the trusted AI Assistant and Agent platform which focuses on enterprise readiness for mission-critical applications. For more [details](../providers/vectara.ipynb).


[Vectara](https://vectara.com/)  provides several tools that can be used with LangChain.
- **VectaraSearch**: For semantic search over your corpus
- **VectaraRAG**: For generating summaries using RAG
- **VectaraIngest**: For ingesting documents into your corpus
- **VectaraAddFiles**: For uploading the files


## Setup

To use the `Vectara Tools` you first need to install the partner package.


In [ ]:
!uv pip install -U pip && uv pip install -qU langchain-vectara langgraph

# Getting Started

To get started, use the following steps:
1. If you don't already have one, [Sign up](https://www.vectara.com/integrations/langchain) for your free Vectara trial.
2. Within your account you can create one or more corpora. Each corpus represents an area that stores text data upon ingest from input documents. To create a corpus, use the **"Create Corpus"** button. You then provide a name to your corpus as well as a description. Optionally you can define filtering attributes and apply some advanced options. If you click on your created corpus, you can see its name and corpus ID right on the top.
3. Next you'll need to create API keys to access the corpus. Click on the **"Access Control"** tab in the corpus view and then the **"Create API Key"** button. Give your key a name, and choose whether you want query-only or query+index for your key. Click "Create" and you now have an active API key. Keep this key confidential. 

To use LangChain with Vectara, you'll need to have these two values: `corpus_key` and `api_key`.
You can provide `VECTARA_API_KEY` to LangChain in two ways:

## Instantiation

1. Include in your environment these two variables: `VECTARA_API_KEY`.

   For example, you can set these variables using os.environ and getpass as follows:

```python
import os
import getpass

os.environ["VECTARA_API_KEY"] = getpass.getpass("Vectara API Key:")
```

2. Add them to the `Vectara` vectorstore constructor:

```python
vectara = Vectara(
    vectara_api_key=vectara_api_key
)
```

In this notebook we assume they are provided in the environment.

In [4]:
import os

os.environ["VECTARA_API_KEY"] = "<VECTARA_API_KEY>"
os.environ["VECTARA_CORPUS_KEY"] = "<VECTARA_CORPUS_KEY>"
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

from langchain_vectara import Vectara
from langchain_vectara.tools import (
    VectaraAddFiles,
    VectaraIngest,
    VectaraRAG,
    VectaraSearch,
)
from langchain_vectara.vectorstores import (
    ChainReranker,
    CorpusConfig,
    CustomerSpecificReranker,
    File,
    GenerationConfig,
    MmrReranker,
    SearchConfig,
    VectaraQueryConfig,
)

vectara = Vectara(vectara_api_key=os.getenv("VECTARA_API_KEY"))

First we load the state-of-the-union text into Vectara.

Note that we use the `VectaraAddFiles` tool which does not require any local processing or chunking - Vectara receives the file content and performs all the necessary pre-processing, chunking and embedding of the file into its knowledge store.

In this case it uses a .txt file but the same works for many other [file types](https://docs.vectara.com/docs/api-reference/indexing-apis/file-upload/file-upload-filetypes).

In [5]:
corpus_key = os.getenv("VECTARA_CORPUS_KEY")

add_files_tool = VectaraAddFiles(
    name="add_files_tool",
    description="Upload files about state of the union",
    vectorstore=vectara,
    corpus_key=corpus_key,
)

file_obj = File(
    file_path="../document_loaders/example_data/state_of_the_union.txt",
    metadata={"source": "text_file"},
)
add_files_tool.run({"files": [file_obj]})

'Successfully uploaded 1 files to Vectara corpus test-langchain with IDs: state_of_the_union.txt'

## Vectara RAG (retrieval augmented generation)

We now create a `VectaraQueryConfig` object to control the retrieval and summarization options:
* We enable summarization, specifying we would like the LLM to pick the top 7 matching chunks and respond in English

Using this configuration, let's create a LangChain tool `VectaraRAG` object that encpasulates the full Vectara RAG pipeline:

In [33]:
generation_config = GenerationConfig(
    max_used_search_results=7,
    response_language="eng",
    generation_preset_name="vectara-summary-ext-24-05-med-omni",
    enable_factual_consistency_score=True,
)
search_config = SearchConfig(
    corpora=[CorpusConfig(corpus_key=corpus_key)],
    limit=25,
    reranker=ChainReranker(
        rerankers=[
            CustomerSpecificReranker(reranker_id="rnk_272725719", limit=100),
            MmrReranker(diversity_bias=0.2, limit=100),
        ]
    ),
)

config = VectaraQueryConfig(
    search=search_config,
    generation=generation_config,
)

query_str = "what did Biden say?"

vectara_rag_tool = VectaraRAG(
    name="rag-tool",
    description="Get answers about state of the union",
    vectorstore=vectara,
    corpus_key=corpus_key,
    config=config,
)

## Invocation

In [32]:
vectara_rag_tool.run(query_str)

'{\n  "summary": "President Biden discussed several key topics in his recent statements. He emphasized the importance of keeping schools open and noted that with a high vaccination rate and reduced hospitalizations, most Americans can safely return to normal activities [1]. He addressed the need to hold social media platforms accountable for their impact on children and called for stronger privacy protections and mental health services [2]. Biden also announced measures against Russia, including preventing its central bank from defending the Ruble and targeting Russian oligarchs\' assets, as well as closing American airspace to Russian flights [3], [7]. Additionally, he reaffirmed the need to protect women\'s rights, particularly the right to choose as affirmed in Roe v. Wade [5].",\n  "factual_consistency_score": 0.5415039\n}'

## Vectara as a langchain retreiver

The `VectaraSearch` tool can be used just as a retriever. 

In this case, it behaves just like any other LangChain retriever. The main use of this mode is for semantic search, and in this case we disable summarization:

In [8]:
search_config = SearchConfig(
    corpora=[CorpusConfig(corpus_key=corpus_key)],
    limit=25,
    reranker=ChainReranker(
        rerankers=[
            CustomerSpecificReranker(reranker_id="rnk_272725719", limit=100),
            MmrReranker(diversity_bias=0.2, limit=100),
        ]
    ),
)

search_tool = VectaraSearch(
    name="Search tool",
    description="Search for information about state of the union",
    vectorstore=vectara,
    corpus_key=corpus_key,
    search_config=search_config,
)

search_tool.run({"query": "What did Biden say?"})

'[\n  {\n    "index": 0,\n    "content": "The vast majority of federal workers will once again work in person. Our schools are open. Let\\u2019s keep it that way. Our kids need to be in school. And with 75% of adult Americans fully vaccinated and hospitalizations down by 77%, most Americans can remove their masks, return to work, stay in the classroom, and move forward safely.",\n    "source": "text_file",\n    "metadata": {\n      "X-TIKA:Parsed-By": "org.apache.tika.parser.csv.TextAndCSVParser",\n      "Content-Encoding": "UTF-8",\n      "X-TIKA:detectedEncoding": "UTF-8",\n      "X-TIKA:encodingDetector": "UniversalEncodingDetector",\n      "Content-Type": "text/plain; charset=UTF-8",\n      "source": "text_file",\n      "framework": "langchain"\n    },\n    "score": 0.9988395571708679\n  },\n  {\n    "index": 1,\n    "content": "Children were also struggling before the pandemic. Bullying, violence, trauma, and the harms of social media. As Frances Haugen, who is here with us tonigh

## Chaining with Vectara tools

You can chain Vectara tools with other LangChain components. The example shows how to:
- Set up a ChatOpenAI model for additional processing
- Create a custom prompt template for specific summarization needs
- Chain multiple components together using LangChain's Runnable interface
- Process and format the JSON response from Vectara

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableSerializable
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

# Create a prompt template
template = """
Based on the following information from the State of the Union address:

{rag_result}

Please provide a concise summary that focuses on the key points mentioned.
If there are any specific numbers or statistics, be sure to include them.
"""
prompt = ChatPromptTemplate.from_template(template)


# Create a function to get RAG results
def get_rag_result(query: str) -> str:
    result = vectara_rag_tool.run(query)
    result_dict = json.loads(result)
    return result_dict["summary"]


# Create the chain
chain: RunnableSerializable = (
    {"rag_result": get_rag_result} | prompt | llm | StrOutputParser()
)

# Run the chain
chain.invoke("What were the key economic points in Biden's speech?")

"President Biden's State of the Union address highlighted key economic points, including closing the coverage gap and making savings permanent, cutting energy costs by $500 annually through climate change initiatives, and providing tax credits for energy efficiency. He emphasized doubling clean energy production and reducing electric vehicle costs. Biden proposed cutting child care costs, making housing more affordable, and offering Pre-K for young children. He assured that no one earning under $400,000 would face new taxes and emphasized the need for a fair tax system. His plan to fight inflation focuses on lowering costs without reducing wages, increasing domestic production, and closing tax loopholes for the wealthy. Additionally, he advocated for raising the minimum wage, extending the Child Tax Credit, and ensuring fair pay and opportunities for workers."

## Use within an agent

The code below demonstrates how to use Vectara tools with LangChain to create an agent.

In [31]:
import json

from langchain_core.messages import HumanMessage
from langchain_openai.chat_models import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# Set up the tools and LLM
tools = [vectara_rag_tool]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Construct the ReAct agent
agent_executor = create_react_agent(llm, tools)

question = (
    "What is an API key? What is a JWT token? When should I use one or the other?"
)
input_data = {"messages": [HumanMessage(content=question)]}


agent_executor.invoke(input_data)

{'messages': [HumanMessage(content='What is an API key? What is a JWT token? When should I use one or the other?', additional_kwargs={}, response_metadata={}, id='2d0d23c4-ca03-4164-8417-232ce12b47df'),
  AIMessage(content="An API key and a JWT (JSON Web Token) are both methods used for authentication and authorization in web applications, but they serve different purposes and have different characteristics.\n\n### API Key\n- **Definition**: An API key is a unique identifier used to authenticate a client making requests to an API. It is typically a long string of characters that is passed along with the API request.\n- **Usage**: API keys are often used for simple authentication scenarios where the client needs to be identified, but there is no need for complex user authentication or session management.\n- **Security**: API keys can be less secure than other methods because they are often static and can be easily exposed if not handled properly. They should be kept secret and not inclu

## VectaraIngest Example

The `VectaraIngest` tool allows you to directly ingest text content into your Vectara corpus. This is useful when you have text content that you want to add to your corpus without having to create a file first.

Here's an example of how to use it:

In [25]:
ingest_tool = VectaraIngest(
    name="ingest_tool",
    description="Add new documents about planets",
    vectorstore=vectara,
    corpus_key=corpus_key,
)

# Test ingest functionality
texts = ["Mars is a red planet.", "Venus has a thick atmosphere."]

metadatas = [{"type": "planet Mars"}, {"type": "planet Venus"}]

ingest_tool.run(
    {
        "texts": texts,
        "metadatas": metadatas,
        "doc_metadata": {"test_case": "langchain tool"},
    }
)

'Successfully ingested 2 documents into Vectara corpus test-langchain with IDs: 0de5bbb6c6f0ac632c8d6cda43f02929, 5021e73c9a9128b05c7a94b299744190'

## API reference

For details checkout implementation of Vectara [tools](https://github.com/vectara/langchain-vectara/blob/main/libs/vectara/langchain_vectara/tools.py).